In [1]:
from papers import *
from ads import *
# imports


In [2]:
my_p = PaperExt(ads_id="2015ApJ...799..103Z")

In [4]:
print(my_p.bib_text)

@ARTICLE{2015ApJ...799..103Z,
       author = {{Zhang}, Gao-Yuan and {Chen}, Yang and {Su}, Yang and {Zhou}, Xin and
         {Pannuti}, Thomas G. and {Zhou}, Ping},
        title = "{The Metal-enriched Thermal Composite Supernova Remnant Kesteven 41 (G337.8-0.1) in a Molecular Environment}",
      journal = {\apj},
     keywords = {ISM: individual objects: G337.8─0.1 \&amp;equals Kesteven 41, ISM: molecules, ISM: supernova remnants, Astrophysics - High Energy Astrophysical Phenomena},
         year = "2015",
        month = "Jan",
       volume = {799},
       number = {1},
          eid = {103},
        pages = {103},
          doi = {10.1088/0004-637X/799/1/103},
archivePrefix = {arXiv},
       eprint = {1502.02668},
 primaryClass = {astro-ph.HE},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2015ApJ...799..103Z},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}


In [5]:
test_lp = ads_get_ref_cit("https://ui.adsabs.harvard.edu/abs/2015ApJ...799..103Z/citations")
test_lp.summary()

The number of current page is  14
0 Multipole Analysis of Radio Continuum Images of Supernova Remnants: Comparison of Type Ia and Core Collapse
1 Nonequilibrium Ionization in Mixed-morphology Supernova Remnants
2 A Suzaku X-ray study of the mixed-morphology supernova remnant Kes 69 and searching for its gamma-ray counterpart
3 Unidentified γ-ray emission towards the SNR Kes 41 revisited
4 Natal molecular cloud of SNR Kes 41. Complete characterisation
5 Eigenvalue Method for NEI Unit in FLASH Code
6 The Age Evolution of the Radio Morphology of Supernova Remnants
7 A revisited study of the unidentified $\gamma$-ray emission towards the SNR Kes 41
8 Asymmetric Type-Ia supernova origin of W49B as revealed from spatially resolved X-ray spectroscopic study
9 The gas-to-extinction ratio and the gas distribution in the Galaxy
10 CTIO, ROSAT HRI, and Chandra ACIS Observations of the Archetypical Mixed-morphology Supernova Remnant W28 (G6.4-0.1)
11 The GeV Gamma-Ray Emission Detected by Fermi-LA

In [6]:
with open("tmp_list.pkl", "wb") as f:
    pck.dump(test_lp, f)

In [8]:
print(test_lp.list_paper[0].bib_text)

@ARTICLE{2019arXiv190911803R,
       author = {{Ranasinghe}, Sujith and {Leahy}, Denis},
        title = "{Multipole Analysis of Radio Continuum Images of Supernova Remnants: Comparison of Type Ia and Core Collapse}",
      journal = {arXiv e-prints},
     keywords = {Astrophysics - High Energy Astrophysical Phenomena},
         year = "2019",
        month = "Sep",
          eid = {arXiv:1909.11803},
        pages = {arXiv:1909.11803},
archivePrefix = {arXiv},
       eprint = {1909.11803},
 primaryClass = {astro-ph.HE},
       adsurl = {https://ui.adsabs.harvard.edu/abs/2019arXiv190911803R},
      adsnote = {Provided by the SAO/NASA Astrophysics Data System}
}


In [26]:
driver_exec_path = r"/usr/local/bin/chromedriver"

class ListPapersExt(ListPapers):
    """Extend the ListPapers to ADS fields"""
    def __init__(self, *args, **kwargs):
        if(type(args[0]) is ListPapers):
            attr_dict = args[0].__dict__
            for kk in attr_dict:
                setattr(self, kk, attr_dict[kk])
        else:
            super().__init__(*args, **kwargs)
            
    def filter_std_key_words(self, std_kw, exclude=False):
        """Make a new ListPaperExt object with the keywords in std_kw_list.
        Or exclude them. 
        std_kw:   accept list of str
                  or str
        """
        if (type(std_kw) is str):
            std_kw = [std_kw]
        if (not exclude):
            new_list = list([pp for pp in self.list_paper if self._std_kw_contain(pp, std_kw)])
        else:
            new_list = list([pp for pp in self.list_paper if not self._std_kw_contain(pp, std_kw)])
        try:
            return ListPapersExt(new_list, key_words=self.key_words, exclude_key_words=self.exclude_key_words, boost=self._boost)
        except:
            try:
                return ListPapersExt(new_list, key_words=self.key_words, boost=self._boost)
            except AttributeError:
                return ListPapersExt(new_list)
        raise
        
    def _std_kw_contain(self, pp, kw):
        """Return True if a paper pp contains any of kw as std_keywords."""
        # kw is a list
        for kk in kw:
            if (kk in pp.std_keywords):
                return True
        return False
    
    def all_std_keywords(self):
        """Show all the std key words from ADS"""
        try:
            self.std_kw_list
        except AttributeError:
            self.std_kw_list = []
        for pp in self.list_paper:
            #std_kw = [cc.strip() for cc in pp.std_keywords]
            for ss in pp.std_keywords:
                if (ss not in self.std_kw_list):
                    self.std_kw_list.append(ss)
        return self.std_kw_list
            
    def summary(self):
        """Change the way it summarizes."""
        super().summary()
        print("Common keywords: ")
        print(self.all_std_keywords())
        
    @staticmethod
    def ext_ads_get_ref_cit(url, nap=5.):
        """Get the reference list or citation list from ADS."""
        wait_for_response = 10. #sec
        try:
            t0 = time.time()
            options = Options()
            options.add_argument(f'user-agent='+random.choice(user_agent_list))
            driver = webdriver.Chrome(chrome_options=options, executable_path=driver_exec_path)
            #driver = webdriver.Chrome(executable_path=r"/usr/local/bin/chromedriver")
            driver.get(url)
            element = WebDriverWait(driver, 50).until(
                EC.presence_of_element_located((By.CLASS_NAME, 's-results-title'))
                )
            #el_per_page = driver.find_element_by_id("per-page-select")
            #el_per_page_500 = el_per_page.find_element_by_xpath("//select[@name='per-page-select']/option[text()='500']")
            #el_per_page_500.click()
            ## Wait for respond
            #time.sleep(wait_for_response)
            source_code = driver.page_source
            soup = BeautifulSoup(source_code, "html.parser")

            bibcode_list_sp = soup.findAll("a", {"aria-label":"bibcode"})
            # Maximum is 500
            page_loop = False
            if (len(bibcode_list_sp) >= 25):
                #print("Warning: this page has at least 500 papers, which means the list could be larger than that.")
                page_loop = True

            print("The number of current page is ", len(bibcode_list_sp))

            paper_list = []
            count = 0
            for bb in bibcode_list_sp:
                tt = time.time()
                paper1 = PaperExt(ads_id=bb.get_text().strip(), nap=pl_nap(tt-t0, nap))
                paper_list.append(paper1)
                print(count, paper1.title)
                count += 1

            count_page = 0
            while(page_loop):
                count_page += 1
                print("Page read ", count_page)
                np_button = driver.find_element(By.XPATH, "//li/a[@class='page-control next-page']")
                try:
                    np_button.click()
                except WebDriverException:
                    break
                time.sleep(wait_for_response)
                source_code = driver.page_source
                soup = BeautifulSoup(source_code, "html.parser")
                bibcode_list_sp = soup.findAll("a", {"aria-label":"bibcode"})
                print("The number of current page is ", len(bibcode_list_sp))
                for bb in bibcode_list_sp:
                    tt = time.time()
                    paper1 = PaperExt(ads_id=bb.get_text().strip(), nap=pl_nap(tt-t0, nap))
                    paper_list.append(paper1)
                    print(count, paper1.title)
                    count += 1
                if (count_page>400):
                    print("Warning: Pages exceed 400. Citation is over 10000. I quit.")
                    break
        except:
            raise
        finally:
            driver.quit()

        print("Sleep...")
        time.sleep(random.random()*nap)
        print("Awake...")
        return ListPapersExt(paper_list)
    
    def export_bib(self, file_name="cite.bib", change_index=False):
        """Based on PaperExt"""
        with open(file_name, "w") as f:
            f.write("%% This file is created by 'export_bib' function in ListPapersExt module.\n\n")
            for pp in self.list_paper:
                if(type(pp) is not PaperExt):
                    print("Warning: paper may not contain any citation text.")
                    continue
                if(pp.bib_text==""):
                    print("Warning: no bib information available.")
                f.write(pp.bib_text)
                f.write("\n\n")
        return
    
    @staticmethod
    def _change_index(*args, to_format="author+year"):
        """Index change to other format.
        Args[0] is the text, args[1] is bibcode, args[2] is authors, args[3] is year"""
        if(to_format=="arxiv"):
            pass
        else:
            pass
        
    def __add__(self, other_list):
        """Enable add."""
        return ListPapersExt(self.list_paper+other_list.list_paper)

    @staticmethod
    def dig_deep(pp, nap=5., deep=2, only=False):
        """Loop the search for referencs and citations"""
        if (pp is not PaperExt):
            raise TypeError("Paper need to be PaperExt!")
            
        if (only=="reference"):
            return ListPapersExt.ext_ext_ads_get_ref_cit(pp.referenc_url, nap=nap)
        elif (only=="citation"):
            return ListPapersExt.ext_ext_ads_get_ref_cit(pp.referenc_url, nap=nap)
        else:
            pass

In [27]:
lp_ads = ListPapersExt(test_lp)

In [28]:
lp_ads.all_std_keywords()

['Astrophysics - High Energy Astrophysical Phenomena',
 'hydrodynamics',
 'ISM: molecules',
 'ISM: structure',
 'methods: numerical',
 'supernovae: general',
 'ISM: individual objects: Kesteven 69 (G21.8 - 0.6)',
 'ISM: supernova remnants',
 'X-rays: ISM',
 'gamma-rays: ISM',
 'ISM: individual objects: Kes 41',
 'gamma rays: ISM',
 'atomic data',
 'atomic processes',
 'plasmas',
 'Astrophysics - Instrumentation and Methods for Astrophysics',
 'ISM: individual objects: W49B',
 'nuclear reactions',
 'nucleosynthesis',
 'abundances',
 'white dwarfs',
 'dust',
 'extinction',
 'planetary nebulae: general',
 'X-rays: binaries',
 'Astrophysics - Astrophysics of Galaxies',
 'X-rays: individual: W28 (G6.4─0.1 CXOU J175857.55─233400.3 3XMM J180058.5─232735',
 'X-rays: stars',
 'acceleration of particles',
 'gamma rays: observations',
 'supernova remnants',
 'ISM: individual (Kes 41)',
 'ISM: individual objects: G33.6+0.1 &amp; Kes 79',
 'pulsars: individual: PSR J1852+0040']

In [29]:
lp_new = lp_ads.filter_std_key_words("Astrophysics - High Energy Astrophysical Phenomena")
lp_new.summary()

Number of papers: 11
Subjects: 
['']
Key words: 

Exclude key words: 
Total scores:

Common keywords: 
['Astrophysics - High Energy Astrophysical Phenomena', 'hydrodynamics', 'ISM: molecules', 'ISM: structure', 'methods: numerical', 'supernovae: general', 'ISM: individual objects: Kesteven 69 (G21.8 - 0.6)', 'ISM: supernova remnants', 'X-rays: ISM', 'gamma-rays: ISM', 'ISM: individual objects: Kes 41', 'gamma rays: ISM', 'atomic data', 'atomic processes', 'plasmas', 'Astrophysics - Instrumentation and Methods for Astrophysics', 'ISM: individual objects: W49B', 'nuclear reactions', 'nucleosynthesis', 'abundances', 'white dwarfs', 'X-rays: individual: W28 (G6.4─0.1 CXOU J175857.55─233400.3 3XMM J180058.5─232735', 'X-rays: stars', 'ISM: individual objects: G33.6+0.1 &amp; Kes 79', 'pulsars: individual: PSR J1852+0040', 'acceleration of particles']


In [35]:
lp_add = lp_new + lp_ads
lp_add.list_paper

AttributeError: 'PaperExt' object has no attribute 'ads_link'

In [19]:
print(lp_ads[3].std_keywords)

['ISM: supernova remnants', 'ISM: individual objects: Kes 41', 'gamma rays: ISM']
